# Coding exercise to generate directly JS scripts for Plotly

In [1]:
from datetime import datetime as dt

from pandas_datareader import data as pdr
import pandas as pd
import numpy as np
import os

In [2]:
tickers=['^GSPC','^N225','^FCHI','^SMSI']
years=[2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]

df=[]

for year in years:
    startdate=dt(year,1,1)
    enddate=dt(year,12,31)
    df_temp=pdr.get_data_yahoo(tickers, start=startdate, end=enddate)['Close']
    for ticker in tickers:
        perf=((df_temp[ticker].dropna().iloc[-1]/df_temp[ticker].dropna().iloc[0])-1)*100
        df.append({'Country':ticker, 'Year':year,'Performance':perf})

df=pd.DataFrame(df, columns=('Country','Year','Performance'))
df.head()

,Country,Year,Performance
0,^GSPC,2010,11.001865
1,^N225,2010,-3.996983
2,^FCHI,2010,-4.066048
3,^SMSI,2010,-20.487817
4,^GSPC,2011,-1.121972


In [3]:
df=df.replace(to_replace=['^GSPC','^N225','^FCHI','^SMSI'], value=['USA','JPN','FRA','CHE'])

In [4]:
df.head()

,Country,Year,Performance
0,USA,2010,11.001865
1,JPN,2010,-3.996983
2,FRA,2010,-4.066048
3,CHE,2010,-20.487817
4,USA,2011,-1.121972


In [5]:
countries = ['USA','JPN','FRA','CHE']

In [11]:
years=""
ret=""
d={}

for country in countries:
    temp=df[df.Country==country]
    for index,row in temp.iterrows():
        years=years+"'"+str(row["Year"])+"',"
        ret=ret+"'"+str(round(row["Performance"],2))+"',"

    d["script"+str(country)]= """var stock"""+str(country)+""" = {
            x: ["""+ years[:-1] + """],
            y: ["""+ ret[:-1] +"""],
            mode: 'lines+markers',
            name: 'Stock """+ str(country)+"""',
            line: {
                color: 'rgb(28,146,46)',
                width: 4,
                shape: 'spline'
            },
            marker: {
                color: 'rgb(28,146,46)',
                size: 12
            }            

            };"""


In [15]:
final=d["scriptFRA"]+d["scriptUSA"]+d["scriptCHE"]+d["scriptJPN"]+"""var data = [stockFRA, stockUSA, stockCHE, stockJPN];

var layout ={
            title: 'Compared stock performances',
            xaxis: {
            tickmode: 'array',
            tickvals: ["""+years+"""]
            }
            };

Plotly.newPlot('myViz', data, layout);"""

In [16]:
file=open("test.js", "w")
file.write(final)
file.close()